In [1]:
from transformers import AutoTokenizer, AutoModel
import torch

model_path = '/root/autodl-tmp/github/FlagEmbedding/projects/model_output/icl_finetune_round1/merged_model_lora_epoch_1'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModel.from_pretrained(model_path)

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model.half()
model.to(device)
model.eval()

txt = 'Mo salah is a football player'
with torch.no_grad():
    ids = tokenizer(txt, padding=True, truncation=True,
                             return_tensors="pt")
    ids = {k: v.to(device) for k, v in ids.items()}
    outputs = model(**ids)
    print(outputs)

del model, tokenizer
import gc
gc.collect()
torch.cuda.empty_cache()

/root/miniconda3/envs/kaggle_nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 6/6 [00:00<00:00,  9.00it/s]
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


BaseModelOutputWithPast(last_hidden_state=tensor([[[-1.4258,  1.0342, -2.1250,  ..., -0.2620, -2.2480,  3.0020],
         [ 2.5508, -4.4492, -3.1914,  ..., -2.8965,  8.7344,  4.0977],
         [ 3.7617, -4.8633,  2.9727,  ...,  5.4219, -5.5234,  7.2656],
         ...,
         [-3.0000, -0.8760,  2.4863,  ..., -6.8633, -1.4434,  0.9424],
         [-2.7480,  1.1318, -1.0654,  ..., -1.8369,  6.4766,  2.7520],
         [-0.4766, -1.7861,  3.1816,  ...,  2.6895,  2.4883,  1.3594]]],
       device='cuda:0', dtype=torch.float16), past_key_values=None, hidden_states=None, attentions=None)


In [2]:
from transformers import AutoTokenizer, AutoModel
import torch
from peft import PeftModel

model_path = 'BAAI/bge-en-icl'
lora_path = '/root/autodl-tmp/github/FlagEmbedding/projects/model_output/icl_finetune_round1/lora_epoch_1'

print("Loading LoRA model...")
tokenizer = AutoTokenizer.from_pretrained(lora_path)
model = AutoModel.from_pretrained(model_path, quantization_config=None)
model = PeftModel.from_pretrained(model, lora_path, is_trainable=False)

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model.half()
model.to(device)
model.eval()

txt = 'Mo salah is a football player'
with torch.no_grad():
    ids = tokenizer(txt, padding=True, truncation=True,
                             return_tensors="pt")
    ids = {k: v.to(device) for k, v in ids.items()}
    outputs = model(**ids)
    print(outputs)

del model, tokenizer
import gc
gc.collect()
torch.cuda.empty_cache()

Loading LoRA model...


Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00,  7.65it/s]
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


BaseModelOutputWithPast(last_hidden_state=tensor([[[-1.4258,  1.0342, -2.1230,  ..., -0.2615, -2.2480,  3.0020],
         [ 2.5508, -4.4414, -3.1914,  ..., -2.8984,  8.7500,  4.0938],
         [ 3.7656, -4.8477,  2.9668,  ...,  5.4102, -5.5234,  7.2773],
         ...,
         [-2.9980, -0.8672,  2.4805,  ..., -6.8711, -1.4404,  0.9360],
         [-2.7500,  1.1367, -1.0752,  ..., -1.8486,  6.4688,  2.7383],
         [-0.4780, -1.7852,  3.1836,  ...,  2.6953,  2.4922,  1.3643]]],
       device='cuda:0', dtype=torch.float16), past_key_values=None, hidden_states=None, attentions=None)


In [3]:
from transformers import AutoTokenizer, AutoModel
import torch
from peft import PeftModel

model_path = 'BAAI/bge-en-icl'

print("Loading base model...")
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModel.from_pretrained(model_path, quantization_config=None)

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model.half()
model.to(device)
model.eval()

txt = 'Mo salah is a football player'
with torch.no_grad():
    ids = tokenizer(txt, padding=True, truncation=True,
                             return_tensors="pt")
    ids = {k: v.to(device) for k, v in ids.items()}
    outputs = model(**ids)
    print(outputs)

del model, tokenizer
import gc
gc.collect()
torch.cuda.empty_cache()

Loading base model...


Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00,  7.01it/s]
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


BaseModelOutputWithPast(last_hidden_state=tensor([[[-1.3027,  0.8101, -2.0469,  ..., -0.3611, -2.3027,  3.0195],
         [ 3.0273, -4.6875, -3.6719,  ..., -2.4082,  9.7422,  5.1719],
         [ 5.1914, -5.3438,  4.9180,  ...,  5.4062, -4.8750,  6.8906],
         ...,
         [-2.6816, -1.9219,  1.9717,  ..., -7.4766, -2.0039,  2.1680],
         [-2.8652,  1.4805, -1.3057,  ..., -1.3633,  7.5195,  3.6895],
         [-0.5249, -1.6484,  1.2695,  ...,  2.7910,  1.0635,  3.4199]]],
       device='cuda:0', dtype=torch.float16), past_key_values=None, hidden_states=None, attentions=None)


In [19]:
import os
import json
import faiss
import numpy as np

EMBEDDER_EVAL_DATA_DIR = '/root/autodl-tmp/github/FlagEmbedding/projects/data/embedder_train_eval_data/cross_validation'
corpus_path = os.path.join(EMBEDDER_EVAL_DATA_DIR, "corpus.jsonl")
queries_path = os.path.join(EMBEDDER_EVAL_DATA_DIR, "train_queries.jsonl")
corpus = [json.loads(line)['text'] for line in open(corpus_path, 'r')] # list of strings
queries = [json.loads(line) for line in open(queries_path, 'r')] # list of strings
print(f"Number of corpus: {len(corpus)}")
print(f"Number of queries: {len(queries)}")

dim = 32
doc_embeddings = np.random.randn(len(corpus), dim)
query_embeddings = np.random.randn(len(queries), dim)



Number of corpus: 2587
Number of queries: 3506


In [20]:
corpus[:2]

['Does not know that angles in a triangle sum to 180 degrees',
 'Uses dividing fractions method for multiplying fractions']

In [21]:
queries[:2]

[{'query': 'Question: \\[\n3 \\times 2+4-5\n\\]\nWhere do the brackets need to go to make the answer equal \\( 13 \\) ?\nHint: Use the order of operations to carry out calculations involving powers\nCorrect answer: \\( 3 \\times(2+4)-5 \\)\nWrong answer: Does not need brackets',
  'pos': ['Confuses the order of operations, believes addition comes before multiplication'],
  'neg': [],
  'correct_id': 1672.0,
  'question_id_answer': '0_D',
  'subject_id': 33,
  'construct_id': 856,
  'prompt': "Given a multiple choice math question and a student's incorrect answer choice, identify and retrieve the specific mathematical misconception or error in the student's thinking that led to this wrong answer."},
 {'query': 'Question: Simplify the following, if possible: \\( \\frac{m^{2}+2 m-3}{m-3} \\)\nHint: Simplify an algebraic fraction by factorising the numerator\nCorrect answer: Does not simplify\nWrong answer: \\( m+1 \\)',
  'pos': ['Does not know that to factorise a quadratic expression, to

In [22]:
print("Building index...")
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(doc_embeddings)
distances, indices = index.search(query_embeddings, k=25)
print(f"Distances shape: {distances.shape}, Indices shape: {indices.shape}")
    

Building index...
Distances shape: (3506, 25), Indices shape: (3506, 25)


In [26]:
path = "./test.jsonl"
with open(path, 'w') as f:
    for line, indice in zip(queries, indices):
        new_line = {}
        new_line['query'] = line['query']
        new_line['pos'] = line['pos']
        new_line['neg'] = []
        for id in indice:
            new_line['neg'].append(corpus[id])
        new_line['prompt'] = line['prompt']
        f.write(json.dumps(new_line) + '\n')

In [25]:
new_line

{'query': 'Question: \\[\n3 \\times 2+4-5\n\\]\nWhere do the brackets need to go to make the answer equal \\( 13 \\) ?\nHint: Use the order of operations to carry out calculations involving powers\nCorrect answer: \\( 3 \\times(2+4)-5 \\)\nWrong answer: Does not need brackets',
 'pos': ['Confuses the order of operations, believes addition comes before multiplication'],
 'neg': ['Believes a direct proportion graph does not have to start at the origin',
  'Adds a zero onto the end of units multiplied by units when solving multiplication problems using the grid method',
  'Subtracts rather than multiplies when answering worded problems',
  'Does not know how to find the next term in a sequence',
  'Believes a smaller number of shaded parts represents a smaller fraction',
  'Mixes up squaring and halving',
  'Believes fraction to decimal conversion is done by writing the numerator as a decimal',
  'Confuses right angle triangles and equilateral triangles',
  'Does not increase the denomina

In [16]:
indices[0]

array([1167,    3,  293, 1387, 2528, 1538, 1976, 2127, 1871,  518,  308,
        381,   10, 1660,  632, 2208, 2039, 1868, 1662,  509,  675, 1452,
       1334,  882,  371])